In [3]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric

In [4]:
import os
import pandas as pd
from utils import (
    ESTACIONES_XLSX,
    NCEP_NCAR_XLSX,
    NDEFM_XLSX
)

In [5]:
df_est = pd.read_excel(ESTACIONES_XLSX, sheet_name=1, header=1, parse_dates=['Año hid'])
df_est.head()

,Año hid,San Luis,Miraflores,Pozo Sarmiento,Pie de Medano,Valle Viejo,Caimancito,Casa de Piedra,Potrero del Clavillo,Piedra Grande,Los Sosa,Padilla,La Maria,San Luis Tucuman
0,1946-01-01,NaN,NaN,133.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1947-01-01,NaN,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1948-01-01,NaN,NaN,76.3,NaN,NaN,41.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1949-01-01,NaN,NaN,64.4,NaN,NaN,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-01-01,NaN,NaN,60.0,NaN,NaN,134.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
from datetime import datetime as dt

df_pred = df_est[['Año hid', 'Pozo Sarmiento']].rename(columns={'Año hid':'ds', 'Pozo Sarmiento':'y'})
df_pred = df_pred[df_pred.ds >= dt(1965, 1, 1)]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_absolute_error
import numpy as np

def train_(df_pred, wsize=10, pred_size=10):
    window_size = wsize
    pred_size = pred_size

    errors = []
    
    for i in range(0, df_pred.pipe(len) - window_size - pred_size + 1):
        m = Prophet(growth='linear', weekly_seasonality=False, daily_seasonality=False, n_changepoints=0, )
        df_ = df_pred.iloc[i:i+window_size]
        m.fit(df_)
        future = m.make_future_dataframe(periods=window_size, freq='YS')
        fcst = m.predict(future)
        y = df_pred.iloc[i+window_size:i+window_size+pred_size].y
        y_hat = fcst.iloc[-pred_size:].yhat
        
        err = mean_absolute_error(y, y_hat)
        errors.append(err)
    
    return np.mean(errors), np.std(errors), errors
        
for i in range(5, 15):
    mean_errors, std_errors, errors = train_(df_pred, wsize=i)
    print(f"wsize: {i} => {mean_errors} ({std_errors}) {min(errors)} - {max(errors)}")

# Asi nomas

In [8]:
df_est[['Año hid', 'San Luis Tucuman']].rename(columns={'Año hid':'ds', 'San Luis Tucuman':'y'})
df_pred = df_pred[df_pred.ds >= dt(1973, 1, 1)]

In [9]:
m = Prophet(growth='linear', weekly_seasonality=False, daily_seasonality=False, n_changepoints=0)
m.fit(df_pred)
future = m.make_future_dataframe(periods=10, freq='YS')
fcst = m.predict(future)

In [10]:
fcst[-10:][['yhat']].reset_index(drop=True).to_csv('predictions/simple_prophecy.csv', header=False)